In [1]:
import pandas as pd
import numpy as np

# Load dataset
df = pd.read_excel("../Data/Indian_EV_User_Spending_Behavior.xlsx")
print("Data Shape:", df.shape)
df.head()


Data Shape: (10000, 16)


,User_ID,City,Vehicle_Type,Charging_Station_ID,Session_Start,Session_End,Energy_Consumed_kWh,Cost_per_kWh,Payment_Mode,Subscription_Type,Wallet_Balance,Charger_Type,Month,Year,Total_Cost,Average_Monthly_Spend
0,U00001,Hyderabad,2W,ST979,2024-01-01 00:00:00,2024-01-01 01:00:00,6.12,11.84,UPI,Basic,336.08,Superfast,12,2024,72.46,72.46
1,U00002,Ahmedabad,4W,ST318,2024-01-01 01:00:00,2024-01-01 02:00:00,39.20,6.03,Credit Card,Basic,778.06,Slow,6,2025,236.38,236.38
2,U00003,Delhi,4W,ST198,2024-01-01 02:00:00,2024-01-01 03:00:00,27.11,6.80,UPI,Premium,1593.61,Slow,5,2025,184.35,184.35
3,U00004,Chennai,4W,ST636,2024-01-01 03:00:00,2024-01-01 04:00:00,9.41,6.12,Credit Card,Basic,433.03,Slow,2,2025,57.59,57.59
4,U00005,Delhi,2W,ST764,2024-01-01 04:00:00,2024-01-01 05:00:00,38.95,7.94,Credit Card,Pay-as-you-go,1788.99,Superfast,1,2025,309.26,309.26


In [2]:
df['charging_duration_min'] = (df['Session_End'] - df['Session_Start']).dt.total_seconds() / 60
df['cost_efficiency'] = df['Total_Cost'] / df['Energy_Consumed_kWh']
df['peak_hour_flag'] = df['Session_Start'].dt.hour.between(18, 22).astype(int)
df['day_of_week'] = df['Session_Start'].dt.dayofweek


In [3]:
def get_season(month):
    if month in [12, 1, 2]:
        return "Winter"
    elif month in [3, 4, 5]:
        return "Summer"
    elif month in [6, 7, 8]:
        return "Monsoon"
    else:
        return "Autumn"

df['season'] = df['Month'].apply(get_season)


In [4]:
monthly = df.groupby(['User_ID', 'Month', 'Year']).agg({
    'Energy_Consumed_kWh': ['sum', 'mean'],
    'Total_Cost': ['sum', 'mean'],
    'Wallet_Balance': 'mean',
    'charging_duration_min': 'mean',
    'cost_efficiency': 'mean',
    'peak_hour_flag': 'mean'
}).reset_index()

monthly.columns = [
    'User_ID', 'Month', 'Year',
    'total_kwh', 'avg_kwh',
    'total_cost', 'avg_cost',
    'avg_wallet_balance',
    'avg_session_duration',
    'avg_cost_efficiency',
    'peak_hour_ratio'
]

# Merge user info back
user_static = df[['User_ID', 'City', 'Vehicle_Type', 'Subscription_Type', 'Payment_Mode', 'Charger_Type']].drop_duplicates()
final_df = pd.merge(monthly, user_static, on='User_ID', how='left')
final_df.head()


,User_ID,Month,Year,total_kwh,avg_kwh,total_cost,avg_cost,avg_wallet_balance,avg_session_duration,avg_cost_efficiency,peak_hour_ratio,City,Vehicle_Type,Subscription_Type,Payment_Mode,Charger_Type
0,U00001,12,2024,6.12,6.12,72.46,72.46,336.08,60.0,11.839869,0.0,Hyderabad,2W,Basic,UPI,Superfast
1,U00002,6,2025,39.20,39.20,236.38,236.38,778.06,60.0,6.030102,0.0,Ahmedabad,4W,Basic,Credit Card,Slow
2,U00003,5,2025,27.11,27.11,184.35,184.35,1593.61,60.0,6.800074,0.0,Delhi,4W,Premium,UPI,Slow
3,U00004,2,2025,9.41,9.41,57.59,57.59,433.03,60.0,6.120085,0.0,Chennai,4W,Basic,Credit Card,Slow
4,U00005,1,2025,38.95,38.95,309.26,309.26,1788.99,60.0,7.939923,0.0,Delhi,2W,Pay-as-you-go,Credit Card,Superfast
